# MeteoSwiss app hail crowdsourced reports toolbox

This jupyter notebook contains several functions to work with the crowdsourced hail reports from the MeteoSwiss application

Step 1 downloads the data from the web service, apply some basic filtering and compute additional fields.

Step 2 finds the maximum value of several radar products over different time windows and spatial radius around the report time and location. It creates bash scripts which further call add_rad_var_crowd.py and get_archived_data_JKO.py. The list of variables can be changed in the notebook and the list of time steps and radius can be changed in add_rad_var_crowd.py.

Step 3 and 4 group the radar variables and merge with previous data.

Step 5 can be used to define regions based on population density or Swiss coordinates to further filter the reports (and use the numpy arrays as filters for corresponding cartesian radar variables if needed).

Step 6 is used to create spatio-temporal clusters of reports using the library st-dbscan. This step doesn't require Step 2 to 4 to be run before (step 5 is not necessary unless a focus on a specific region is required.).

Step 7 is used to merge back all the clusters information together with the original reports dataframe. The resulting dataframe is called crowd_consolidated_2015-05-02_2023-10-15.pkl. Some filtering examples are listed at the end of step 7.

### Please note that all scripts work properly (hopefully) but their code are clearly not optimised.
#### This script has been updated by Margaret from October 2024
-

In [1]:
import numpy as np
import pandas as pd
import datetime
import os
from st_dbscan import ST_DBSCAN
import time
import geopandas as gpd
import cv2

## Step 1: download raw data and add filters

In [2]:
# Download the crowdsourced between date_from and date_to, perform data cleaning, add new fields and filters
date_from = "2024-08-29"
date_to = "2024-09-10"

dates = pd.date_range(date_from,date_to,freq='d').strftime('%Y-%m-%d').tolist()

base_url = "https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start="

raw_data = pd.DataFrame()

#We proceed day by day because download is limited to 10 days max
for idx in range(len((dates))-1):
    url = base_url + dates[idx] + "&end=" + dates[idx+1]
    print(url)
    temp = pd.read_csv(url)
    raw_data = pd.concat([raw_data,temp])

    
#remove last column, artifact without data
raw_data = raw_data.iloc[:,:-1]

#remove potential duplicates
raw_data = raw_data.drop_duplicates(subset=["HagelUserId", "x","y", "Time", "Type"])

raw_data = raw_data.rename(columns={'Type': 'size', 'HagelUserId': 'ID'})

convert_dict = {'ID':str, 'x':int, 'y':int, 'size':int, 'CustomTime':str,
                'CustomLocation':str, 'OsVersion':str, 'AppVersion':str}
 
raw_data = raw_data.astype(convert_dict)

#restrict area to Switerzland: (30000,310000,470000,840000) # minCHX,maxCHX,minCHY,maxCHY
raw_data = raw_data.loc[((raw_data['x']>=470000) & (raw_data['x']<840000) & (raw_data['y']>=30000) & (raw_data['y']<310000))]

#remove default location when GPS not active
raw_data = raw_data.loc[~((raw_data['x']==717144) & (raw_data['y']==95751))]
raw_data = raw_data.loc[~((raw_data['x']==537969) & (raw_data['y']==152459))]
raw_data = raw_data.loc[~((raw_data['x']==683472) & (raw_data['y']==247852))]

#convert from unix epoch time format to usual date time
raw_data['Time'] = pd.to_datetime(raw_data['Time']/1000,unit='s',utc=True)
raw_data['SubmissionTime'] = pd.to_datetime(raw_data['SubmissionTime']/1000,unit='s',utc=True)
raw_data['T_diff'] = (raw_data['SubmissionTime'] - raw_data['Time']).dt.total_seconds()

#add size labels and categories
conditions = [
        (raw_data['size'] == 10),(raw_data['size'] == 11),(raw_data['size'] == 12),(raw_data['size'] == 13),
        (raw_data['size'] == 14),(raw_data['size'] == 15),(raw_data['size'] == 16)]

conditions2 = [
        (raw_data['size'] == 10) | (raw_data['size'] == 0),
        (raw_data['size'] == 11) | (raw_data['size'] == 1) | (raw_data['size'] == 12),
        (raw_data['size'] == 2) | (raw_data['size'] == 13),
        (raw_data['size'] == 3) | (raw_data['size'] == 14),
        (raw_data['size'] == 15) | (raw_data['size'] == 16) | (raw_data['size'] == 4)]

# create a list of the values we want to assign for each condition
values = ['No hail', '< coffee bean', 'Coffee bean', '1 CHF', '5 CHF', 'Golf ball', 'Tennis ball']
values2 = ['No hail', '< 10 [mm]', '10 [mm]', '20 [mm]', '30 [mm]', '50 [mm]', '>70 [mm]']
values3 = ['No hail', '<15 [mm]', '15-27 [mm]', '27-37 [mm]', '>37 [mm]']
values4 = [0,1,2,3,4]

# create a new column and use np.select to assign values to it using our lists as arguments
raw_data['size_text'] = np.select(conditions, values, default='Unknown')
raw_data['size_text_new'] = np.select(conditions, values2, default='Unknown')
raw_data['size_range_all'] = np.select(conditions2, values3, default='Unknown')
raw_data['size_range_all_num'] = np.select(conditions2, values4, default=-1)  # Ensure this matches integer type  
# The default='Unknown' ensures there is a value for any unmatched conditions.     


raw_data['day'] = raw_data['Time'].dt.date
raw_data['month'] = raw_data['Time'].dt.to_period('M')
raw_data['year'] = raw_data['Time'].dt.to_period('Y')
raw_data['Timer'] = raw_data['Time'].dt.round("5min")
raw_data['xr'] = round(raw_data['x']/500.0)*500.0
raw_data['yr'] = round(raw_data['y']/500.0)*500.0

raw_data['month_name'] = raw_data['Time'].dt.month_name()
raw_data['month_number'] = raw_data['Time'].dt.month
raw_data['hour'] = raw_data['Time'].dt.hour
raw_data['Time_only'] = raw_data['Time'].dt.time
    
cond_season = [raw_data['month_number'].isin([12,1,2]), raw_data['month_number'].isin([3,4,5]),
                raw_data['month_number'].isin([6,7,8]), raw_data['month_number'].isin([9,10,11])]
val_season = ['DJF', 'MAM', 'JJA', 'SON']
raw_data['season_name'] = np.select(cond_season, val_season, default='Unknown')

# rounding for further grouping
raw_data['Time_30s'] = raw_data['Time'].dt.round("30s")
raw_data['Time_1min'] = raw_data['Time'].dt.round("1min")
raw_data['Time_2min'] = raw_data['Time'].dt.round("2min")
raw_data['Time_1h'] = raw_data['Time'].dt.round("1h")
raw_data['Time_30s'] = raw_data['Time'].dt.round("30s")

raw_data['x_1000m'] = round(raw_data['x'] / 1000) * 1000
raw_data['x_2000m'] = round(raw_data['x'] / 2000) * 2000

raw_data['y_1000m'] = round(raw_data['y'] / 1000) * 1000
raw_data['y_2000m'] = round(raw_data['y'] / 2000) * 2000


#timestamps for radar data
raw_data['Timestamp'] = raw_data['Timer'].dt.strftime('%Y%m%d%H%M%S')
raw_data['Time_m15'] = (raw_data['Time'] - datetime.timedelta(minutes=15)).dt.ceil(freq='5T').dt.strftime('%Y%m%d%H%M%S')
raw_data['Time_p15'] = (raw_data['Time'] + datetime.timedelta(minutes=15)).dt.floor(freq='5T').dt.strftime('%Y%m%d%H%M%S')
raw_data['Time_m10'] = (raw_data['Time'] - datetime.timedelta(minutes=10)).dt.ceil(freq='5T').dt.strftime('%Y%m%d%H%M%S')
raw_data['Time_p10'] = (raw_data['Time'] + datetime.timedelta(minutes=10)).dt.floor(freq='5T').dt.strftime('%Y%m%d%H%M%S')
raw_data['Time_m5'] = (raw_data['Time'] - datetime.timedelta(minutes=5)).dt.ceil(freq='5T').dt.strftime('%Y%m%d%H%M%S')
raw_data['Time_p5'] = (raw_data['Time'] + datetime.timedelta(minutes=5)).dt.floor(freq='5T').dt.strftime('%Y%m%d%H%M%S')

# Flag reports submitted 30 minutes after event (Barras et al., 2019)
raw_data['Flag_30min'] = 0
raw_data.loc[raw_data['T_diff'] > 1800, 'Flag_30min'] = 1


# Flag reports submitted by same user on same day with more than 3 CustomLocation=TRUE (Barras et al., 2019)
temp = raw_data.groupby(['ID','day','CustomLocation'])
raw_data = raw_data.set_index(['ID','day','CustomLocation'])
raw_data['N_day_ID_custom'] = temp.size()
raw_data = raw_data.reset_index()
raw_data['Flag_day_ID_3_custom'] = 0
raw_data.loc[(raw_data['N_day_ID_custom']>3) & (raw_data['CustomLocation']==True),'Flag_day_ID_3_custom'] = 1

# Flag reports if same user submitted smallest and largest size within 2 min (Barras et al., 2019)
temp = raw_data.groupby(['ID','Time_2min'])
raw_data['Flag_2min_ID_S_XL'] = (((temp['size'].transform(lambda x: x.eq(4).any())) & (temp['size'].transform(lambda x: x.isin([0,1]).any())))
                        | ((temp['size'].transform(lambda x: x.eq(16).any())) & (temp['size'].transform(lambda x: x.isin([10,11,12]).any()))))
raw_data['Flag_2min_ID_S_XL'] = raw_data['Flag_2min_ID_S_XL'].astype(int)

# Flag reports if same user submitted 3 reports, including largest size within 1 hour (Barras et al., 2019)
temp = raw_data.groupby(['ID','Time_1h'])
raw_data['Flag_1h_ID_3size_XL'] = (((temp['size'].transform(lambda x: x.eq(4).any())) & (temp['size'].transform(lambda x: x.nunique() > 2)))
                        | ((temp['size'].transform(lambda x: x.eq(16).any())) & (temp['size'].transform(lambda x: x.nunique() > 2))))
raw_data['Flag_1h_ID_3size_XL'] = raw_data['Flag_1h_ID_3size_XL'].astype(int)

# Blacklist flag (Barras et al., 2019)
raw_data['Flag_blacklist'] = 0
raw_data.loc[(raw_data['Flag_day_ID_3_custom']==1) | (raw_data['Flag_2min_ID_S_XL']==1) |
              (raw_data['Flag_1h_ID_3size_XL']==1),'Flag_blacklist'] = 1

# Flag reports if same user submitted more than 4 on same day
temp = raw_data.groupby(['ID','day'])
raw_data = raw_data.set_index(['ID','day'])
raw_data['N_day_ID'] = temp.size()
raw_data = raw_data.reset_index()
raw_data['Flag_N_day_ID_4'] = 0
raw_data.loc[raw_data['N_day_ID']>4,'Flag_N_day_ID_4'] = 1

# Save as csv
raw_data.to_csv('crowd_reports_%s_%s.csv' % (date_from, date_to))

https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-08-29&end=2024-08-30
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-08-30&end=2024-08-31
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-08-31&end=2024-09-01
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-01&end=2024-09-02
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-02&end=2024-09-03
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-03&end=2024-09-04
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-04&end=2024-09-05
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-05&end=2024-09-06
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-06&end=2024-09-07
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/data?start=2024-09-07&end=2024-09-08
https://app-prod-ws.meteoswiss-app.ch/v1/hagel/A8uWrl4da/dat

/tmp/ipykernel_90040/3265106371.py:80: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  raw_data['month'] = raw_data['Time'].dt.to_period('M')
/tmp/ipykernel_90040/3265106371.py:81: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  raw_data['year'] = raw_data['Time'].dt.to_period('Y')
/tmp/ipykernel_90040/3265106371.py:113: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  raw_data['Time_m15'] = (raw_data['Time'] - datetime.timedelta(minutes=15)).dt.ceil(freq='5T').dt.strftime('%Y%m%d%H%M%S')
/tmp/ipykernel_90040/3265106371.py:114: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  raw_data['Time_p15'] = (raw_data['Time'] + datetime.timedelta(minutes=15)).dt.floor(freq='5T').dt.strftime('%Y%m%d%H%M%S')
/tmp/ipykernel_90040/3265106371.py:115: FutureWarning: 'T' is deprecated and will be removed in 

In [6]:
# Timo's filter for 4th dimension clustering by hail size category

# Does not work at the moment!
# Variable names need to be adjusted to match the dataset**

raw_data["time_int"] = raw_data.time_dt.astype("int64")//10**9  
raw_data["time_int"] = raw_data["time_int"] - raw_data["time_int"].min()
X = raw_data[['chx','chy','time_int','size']].values
X = sc.crowd_process.per_category_scaler(X,min_dist_p_category=[dx,dx,d_min*60,d_size]) # km,km, s, #size_categories
dbscan = DBSCAN(eps=1,min_samples=5)
dbscan.fit(X)
raw_data['cluster'] = dbscan.labels_


def per_category_scaler(X,min_dist_p_category=None):
    """Scale each feature in X by the maximum distance in this dimension which should still be considered as the same cluster in DBSCAN. Equivalent to eps parameter in DBSCAN, but separate for each dimension.
    Args:
    X (np.ndarray): Data to be scaled (n_samples, n_features)
    min_dist_p_category (np.array, optional): DBSCAN eps parameter for each dimension Returns:
    X_scaled (np.ndarray): scaled data """

if min_dist_p_category is None:
    min_dist_p_category = np.ones(X.shape[1])

X_scaled = np.zeros(X.shape) 
for i in range(X.shape[1]):
    X_scaled[:,i] = X[:,i]/min_dist_p_category[i]
return X_scaled

AttributeError: 'DataFrame' object has no attribute 'time_dt'

In [3]:
print(raw_data)

                                        ID         day CustomLocation       x  \
0     2F18EA72-F285-48E4-8DCA-D1CF1CF204C9  2024-08-29           True  760477   
1     B719F193-8FDE-4ABA-BBC9-94A346970A5A  2024-08-29          False  696943   
2     551b89e9-d0a5-4292-b3c2-f7b1fcf14117  2024-08-29          False  683823   
3     FF798AED-00D2-43C3-BC19-D10740F6C728  2024-08-29           True  609829   
4     60415511-7c80-4553-8fd5-2617f362c7a8  2024-08-29          False  595579   
...                                    ...         ...            ...     ...   
5034  84895b8e-c679-4f6d-a381-9213814f59d0  2024-09-09          False  684594   
5035  074efa02-5e98-463c-8481-8c01dbc96dab  2024-09-09          False  679648   
5036  f5f5387c-7d9a-4734-b8b4-e76e86cf6d3b  2024-09-09           True  583262   
5037  58160616-f3b5-49ef-a6c7-69aaa309aac0  2024-09-09          False  541960   
5038  58160616-f3b5-49ef-a6c7-69aaa309aac0  2024-09-09          False  541960   

           y               

## Step 2: add radar variables
- Change ls_rad_var depending on your needs
- Create the sbatch files
- They each call add_rad_var_crowd.py where time_1_list and time_2_list can be adapted. Note that time ranges larger than 15 minutes should be added beforehand in raw_data
- Change the .csv filename in add_rad_var_crowd.py L29
- Run the sbatch
- add_rad_var_crowd.py calls prepare_gridded_radar_data_from_zip from get_archived_data.py (change the path if necessary)

In [4]:
#Create sbatchs to add radar variables
#Run the generated sbatchs in command line (call add_rad_var_crowd.py)

ls_rad_var = ['BZC','MZC','CZC','EZC45','EZC50','HZT']
date_from = "2024-08-29"
date_to = "2024-09-10"

with open('crowd_sbatch', 'r') as file:
    # read a list of lines into data
    data = file.readlines()

for rad_var in ls_rad_var:
    data[10] = '#SBATCH --job-name=crowd%s_%s_%s\n' % (rad_var, date_from, date_to)
    data[15] = 'srun python3 add_rad_var_crowd.py %s %s %s\n' % (rad_var, date_from, date_to)
    data[16] = 'cd /users/mryan/Code\n'
    #data[17] = 'find . -type f -name \'%s%s*\' -delete\n' % (rad_var,yr)
    #data[17] = 'find . -type f -name \'%s*\' -delete\n' % (rad_var)

    with open('crowd_sbatch_%s_%s_%s' % (rad_var, date_from, date_to), 'w') as file:
        file.writelines(data)


## Step 3: group all radar variables in a single dataframe
Also compute maximum of radar variable over time window.
The reflectivity filter of (Barras et al., 2019) is equivalent to maxCZC_t15_r4 >= 35 (dBZ)
- Uncomment the list and loop over name_ to concatenate multiple time periods

In [4]:
# Concatenate radar variables in single dataframe

ls_time = ['5', '10', '15']
ls_radius = ['0', '2', '4']
# date1 = ['2015-01-01','2017-01-01','2019-01-01','2021-01-01','2022-01-01','2023-01-01']
# date2 = ['2017-01-01','2019-01-01','2021-01-01','2022-01-01','2023-01-01','2024-01-01']
date_from = "2024-08-29"
date_to = "2024-09-10"
ls_rad_var = ['CZC','BZC','MZC','EZC45','EZC50','HZT']

cr = pd.read_csv('crowd_reports_%s_%s.csv' % (date_from, date_to)).drop(columns=['Unnamed: 0'])
crowd = cr.copy()
for rad_var in ls_rad_var:    
    #df = pd.read_csv('crowd_%s_%s_%s.csv' % (rad_var,date1[0],date2[0])).drop(columns=['Unnamed: 0'])
    # name_ = ['crowd_%s_%s_%s.csv' % (rad_var,d1,d2) for d1, d2 in zip(date1[1:],date2[1:])]
    # for n in name_:
    #     df_ = pd.read_csv(n).drop(columns=['Unnamed: 0'])
    #     df = df.append(df_,ignore_index=True)
    df = pd.read_csv('crowd_%s_%s_%s.csv' % (rad_var,date_from,date_to)).drop(columns=['Unnamed: 0'])

    fields = df.columns.values.tolist()
    matching = sorted([s for s in fields if rad_var in s or s in (["x","y", "Time"])])
    df = df[matching]
    crowd = pd.merge(crowd, df, how='left', on=["x","y", "Time"])

# Compute maximum of radar variable over previous and next n time steps (time window)
# To get reflectivity filter of (Barras et al., 2019): maxCZC_t15_r4 >= 35 (dBZ)
for c in ls_rad_var:
    for r in ls_radius:
        n_ts = 'max%s_ts_r%s' % (c,r)
        crowd = crowd.rename(columns = {'max%s_rad%s_ts' % (c,r): n_ts})
        for t in ls_time:
            n_ = 'max%s_t%s_r%s' % (c,t,r)
            crowd[n_] = crowd[["max%s_rad%s_m%s" % (c,r,t) , "max%s_rad%s_p%s" % (c,r,t)]].max(axis=1)


crowd = crowd.drop_duplicates(subset=["ID", "x","y", "Time", "size"])

# Save the crowdsourced data with radar variables
crowd.to_csv('crowd_complete_%s_%s.csv' % (date_from,date_to))

FileNotFoundError: [Errno 2] No such file or directory: 'crowd_CZC_2024-08-29_2024-09-10.csv'

## Step 4: append the new data to existing one and select relevant fields

In [ ]:
# Concatenate new data (crowd) with existing (old) (change name of df1 if necessary)
old = pd.read_csv('crowd_complete_2015-05-12_2023-08-31.csv').drop(columns=['Unnamed: 0'])
#new = pd.read_csv('crowd_complete_%s_%s.csv' % (date_from,date_to)).drop(columns=['Unnamed: 0'])

new = crowd
old = old.append(new,ignore_index=True)
old = old.drop_duplicates(subset=["ID", "x","y", "Time", "size"])
old.to_csv('crowd_complete_2015-05-12_%s.csv' % (date_to))

In [50]:
t = pd.read_csv('crowd_complete_2015-05-12_2023-10-15.csv')

/tmp/ipykernel_13613/3236999012.py:1: DtypeWarning: Columns (10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  t = pd.read_csv('crowd_complete_2015-05-12_2023-10-15.csv')


In [9]:
z = t[['ID', 'day', 'CustomLocation', 'x', 'y', 'Time',
       'size', 'CustomTime', 'SubmissionTime', 'OsVersion', 'AppVersion',
       'Language', 'T_diff', 'size_text', 'size_text_new',
       'size_range_all', 'size_range_all_num', 'month', 'year', 'Timer',
       'xr', 'yr', 'Time_30s', 'Time_1min', 'Time_2min', 'Time_1h',
       'x_1000m', 'x_2000m', 'y_1000m', 'y_2000m', 'Flag_30min', 'N_day_ID_custom',
       'Flag_day_ID_3_custom', 'Flag_2min_ID_S_XL', 'Flag_1h_ID_3size_XL',
       'Flag_blacklist', 'N_day_ID', 'Flag_N_day_ID_4','Timestamp',
       'maxCZC_ts_r0', 'maxCZC_ts_r2', 'maxCZC_ts_r4',
       'maxCZC_t5_r0','maxCZC_t10_r0', 'maxCZC_t15_r0',
       'maxCZC_t5_r2', 'maxCZC_t10_r2','maxCZC_t15_r2',
       'maxCZC_t5_r4', 'maxCZC_t10_r4', 'maxCZC_t15_r4',
       'maxBZC_ts_r0','maxBZC_ts_r2', 'maxBZC_ts_r4',
       'maxBZC_t5_r0', 'maxBZC_t10_r0','maxBZC_t15_r0',
       'maxBZC_t5_r2', 'maxBZC_t10_r2', 'maxBZC_t15_r2',
       'maxBZC_t5_r4', 'maxBZC_t10_r4', 'maxBZC_t15_r4',
       'maxMZC_ts_r0','maxMZC_ts_r2', 'maxMZC_ts_r4',
       'maxMZC_t5_r0','maxMZC_t10_r0', 'maxMZC_t15_r0',
       'maxMZC_t5_r2', 'maxMZC_t10_r2','maxMZC_t15_r2',
       'maxMZC_t5_r4', 'maxMZC_t10_r4', 'maxMZC_t15_r4',
       'maxEZC45_ts_r0', 'maxEZC45_ts_r2', 'maxEZC45_ts_r4',
       'maxEZC45_t5_r0', 'maxEZC45_t10_r0', 'maxEZC45_t15_r0',
       'maxEZC45_t5_r2', 'maxEZC45_t10_r2', 'maxEZC45_t15_r2',
       'maxEZC45_t5_r4', 'maxEZC45_t10_r4', 'maxEZC45_t15_r4',
       'maxEZC50_ts_r0', 'maxEZC50_ts_r2', 'maxEZC50_ts_r4',
       'maxEZC50_t5_r0', 'maxEZC50_t10_r0', 'maxEZC50_t15_r0',
       'maxEZC50_t5_r2', 'maxEZC50_t10_r2', 'maxEZC50_t15_r2',
       'maxEZC50_t5_r4', 'maxEZC50_t10_r4', 'maxEZC50_t15_r4',
       'maxHZT_ts_r0', 'maxHZT_ts_r2', 'maxHZT_ts_r4',
       'maxHZT_t5_r0', 'maxHZT_t10_r0', 'maxHZT_t15_r0',
       'maxHZT_t5_r2', 'maxHZT_t10_r2', 'maxHZT_t15_r2',
       'maxHZT_t5_r4', 'maxHZT_t10_r4', 'maxHZT_t15_r4']]

In [14]:
z.to_pickle('crowd_complete_2015-05-02_2023-10-15.pkl')

## Step 5: create regions

### Create numpy array mask from population density data (Swiss100)

In [ ]:
pop_file = 'STATPOP2021.csv'
population = pd.read_csv(pop_file,sep=';')
population['chx'] = np.round(population['E_KOORD']/1000)
population['chy'] = np.round(population['N_KOORD']/1000)
pop_sum = population.groupby(['chx','chy']).agg({'B21BTOT': 'sum'})
chx_range = [2484, 2838]
chy_range = [1073, 1299]

chx = np.arange(chx_range[0],chx_range[1])
chy = np.arange(chy_range[0],chy_range[1])

pop = np.zeros((len(chy),len(chx)))
for i in range(len(pop_sum)): #tqdm()
    x_coord = chx==pop_sum.index.get_level_values('chx')[i]
    y_coord = chy==pop_sum.index.get_level_values('chy')[i]
    pop[y_coord, x_coord] = pop_sum.iloc[i]['B21BTOT']

xx, yy = np.meshgrid(chx, chy)

In [ ]:
# At least 100 people per km2
p = np.where(pop < 100, 0, 1)

# Apply 3x3 dilation
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
c = p.astype('uint8')
dilation = cv2.dilate(c,kernel1,iterations = 1)

In [ ]:
# Extend to radar domain

np.save('CH_pop_maskall100d33.npy',dilation)
b = np.zeros((640, 710))
ch_mask = np.load('CH_pop_maskall100d33.npy')
t = np.flip(ch_mask,axis=0)
for x in np.arange(181,181+226):
    for y in np.arange(228,228+354):
        b[x,y] = t[x-181,y-228]

np.save('all100d33_area_mask_extend.npy',b)

### Create numpy array mask from rectangle (ZRH region)

In [ ]:
b = np.zeros((640, 710))
for x in np.arange(215,215+25):
    for y in np.arange(404,404+40):
        b[x,y] = 1

np.save('ZRH_area_mask_extend.npy',b) #660-700; 240-265

### Create shapefile masks from numpy arrays for selecting reports using geopandas

In [3]:
region = 'all100d33'
from PIL import Image
pop_mask = np.load('%s_area_mask_extend.npy' % (region))
t = pop_mask.copy() * 255
n = np.flipud(t)
#n = t
img = Image.fromarray(n)
img = img.convert("L")
img.save('%s_area_mask_extend.tif' % (region))

In [ ]:
from osgeo import gdal, ogr, osr
raster = gdal.Open('%s_area_mask_extend.tif' % (region))
band = raster.GetRasterBand(1)


shp_proj = osr.SpatialReference()
proj = raster.GetProjection()
shp_proj.ImportFromWkt(proj)

output_file = '%s_area_mask_extend.shp' % (region)
call_drive = ogr.GetDriverByName('ESRI Shapefile')
create_shp = call_drive.CreateDataSource(output_file)
shp_layer = create_shp.CreateLayer('layername', srs = shp_proj)
new_field = ogr.FieldDefn(str('ID'), ogr.OFTInteger)
shp_layer.CreateField(new_field)
dst_field = shp_layer.GetLayerDefn().GetFieldIndex("ID_1")

gdal.Polygonize(band, None, shp_layer, dst_field , [], callback=None)
del create_shp
del raster

### Swiss100 region

In [ ]:
output_file = 'all100d33_area_mask_extend.shp'
regions = gpd.read_file(output_file)
mask = ((regions.area > 200) & (regions.area < 100000))
all100d33_area_temp = regions.loc[mask]
ind_list = [0,2,3]
all100d33_area = all100d33_area_temp.iloc[ind_list].reset_index().drop(columns=['ID'])

### ZRH region

In [4]:
output_file = 'ZRH_area_mask_extend.shp'
ZRH_area = gpd.read_file(output_file)
ZRH_area = ZRH_area.drop([1])

## Step 6: create clusters

### Select reports according to region

In [ ]:
reports = pd.read_pickle('crowd_complete_2015-05-02_2023-10-15.pkl')

# Remove reports with time delay > 30 min, in the blacklist or made by users with 4 reports or more per day
reports = reports.loc[(reports['Flag_30min']==0) & (reports['Flag_blacklist']==0) & (reports['Flag_N_day_ID_4']==0)]

# Remove "no hail" reports
reports = cr.loc[~((cr['size']==0) | (cr['size']==10))].copy()

# Transform reports coordinates into geopandas points
gdf2 = gpd.GeoDataFrame(reports[['ID','x','y','Time','size','hour']], geometry=gpd.points_from_xy(reports.x_1000m/1000 - 255, reports.y_1000m/1000 + 160))

# Select reports only within area
points_within = gpd.sjoin(gdf2, all100d33_area, predicate='within')

In [ ]:
#points_within = points_within.loc[points_within['size'].isin([12,13,14,15,16])]
pts = points_within[['ID','x','y','Time','size','hour']]
#pts = points_within[['ID_left','x','y','Time','size', 'hour']]
#pts = pts.rename(columns={'ID_left': 'ID'})

In [ ]:
df_filt = reports.merge(pts, on=["x","y","Time",'ID','size'], how='left', indicator=True)
df = df_filt.loc[(df_filt['_merge'] == 'both') & (df_filt['Time_only'] >= datetime.time(6,0)) & (df_filt['Time_only'] <= datetime.time(21,15))]
#df = df[['ID','x','y','Time','size','year']]

### Create clusters

In [ ]:
#split by year to run clustering algorithm
years = df['year'].unique().tolist()

df_dict = {elem : pd.DataFrame() for elem in years}

for key in df_dict.keys():
    df_dict[key] = df[:][df.year == key]

#list of parameters for clustering algorithm
list_eps_d = [8,12,16]
list_min_s = [5]
list_eps_t = [480,720,960]
area = 'all100d33_2' #ZRH_2
n_ = ['','_large']

In [ ]:
#stats = []
path = 'C:\\Users\Jérôme\Documents\PhD\Crowdsourced data\\analysis\clusters\\'

size_all = df['size'].unique().tolist()
df = df[['ID','x','y','Time','size','year']]
    
for n in n_:
    name = area + n
    if n == '_nomin':
        sizes = [12,13,14,15,16]
    if n == '_large':
        sizes = [13,14,15,16]
    if n == '':
        sizes = size_all
    temp = df.loc[df['size'].isin(sizes)]
    
    years = temp['year'].unique().tolist()

    df_dict = {elem : pd.DataFrame() for elem in years}

    for key in df_dict.keys():
        df_dict[key] = temp[:][temp.year == key]
    
    for year in years:
        data = df_dict[year]
        data['x_km'] = data['x']/1000
        data['y_km'] = data['y']/1000
        data['Time_int'] = data['Time'].astype('int64')/10**9 #time in seconds
        data['Time_int'] = data['Time_int'] - data['Time_int'].min()
        X_temp = data[['x_km','y_km','Time_int']].copy()
        cols = X_temp.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        X_temp = X_temp[cols]
        X = X_temp.to_numpy(copy=True)

        for eps_d, eps_t in zip(list_eps_d,list_eps_t):
                for min_s in list_min_s:
                    t0 = time.time()
                    st_dbscan = ST_DBSCAN(eps1 = eps_d, eps2 = eps_t, min_samples = min_s) # eps1 = distance in km, eps2 = time in seconds, min_samples = min number of reports to create cluster
                    st_dbscan.fit(X)
                    labels = st_dbscan.labels
                    X_temp['labels'] = labels
                    n = len(set(labels))

                    # Number of clusters in labels, ignoring noise if present.
                    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
                    n_noise_ = list(labels).count(-1)
                    t1 = time.time()
                    total = t1-t0
                    print("Total time: %f" % total)
                    crowd_clust = pd.merge(data, X_temp, how='left', on=["x_km","y_km","Time_int"])
                    crowd_clust.to_pickle(path + 'st_dbscan_%s_%s_%skm_%ss_%sc.pkl' % (year, name, eps_d, eps_t, min_s))
                    #stats.append({'year': year, 'eps_t': eps_t, 'eps_d': eps_d, 'min_size': min_s, 'n_clusters': n_clusters_, 'n_noise': n_noise_, 'time': total})
            
# st = pd.DataFrame.from_dict(stats)
# st.to_csv('st_dbscan_%s_stats.csv' % (area))

In [ ]:
# merge the yearly files

path = 'C:\\Users\Jérôme\Documents\PhD\Crowdsourced data\\analysis\clusters\\'

for n in n_:
    name = area + n
    for eps_d, eps_t in zip(list_eps_d,list_eps_t):
        for min_s in list_min_s:
            data = {}
            for file in os.listdir(path):
                 if file.endswith('%s_%skm_%ss_%sc.pkl' % (name, eps_d, eps_t, min_s)):
                    with open(path+file, 'rb') as f:
                        data[file] = pd.read_pickle(f)

            temp = pd.concat(data.values(), ignore_index=True).drop_duplicates()
            temp.to_pickle('st_dbscan_%s_%skm_%ss_%sc.pkl' % (name, eps_d, eps_t, min_s))

In [ ]:
#Merge clusters data back with reports data
for n in n_:
    name = area + n
    for eps_d, eps_t in zip(list_eps_d,list_eps_t):
        clusters = pd.read_pickle('st_dbscan_%s_%skm_%ss_5c.pkl' % (name,eps_d,eps_t))
        clusters['Time'] = pd.to_datetime(clusters['Time'], format='%Y-%m-%d %H:%M:%S')
        clusters['clust_no'] = 0
        clusters.loc[clusters['labels'] == -1,'clust_no'] = 1
        tot = pd.merge(reports, clusters, how='inner', on=["x","y","Time",'ID','size'])
        tot.to_pickle('crowdsourced_data_%s_%skm_%ss_5c.pkl' % (name,eps_d,eps_t))

## Step 7: create consolidated dataframe with all reports and clusters information

In [9]:
z = pd.read_pickle('crowd_complete_2015-05-02_2023-10-15.pkl')
z['Time'] = pd.to_datetime(z['Time'])

ls_cl = ['8km_480s','12km_720s','16km_960s']

nm_list = ['all100d33_2','ZRH_2','all100d33_2_large','ZRH_2_large']
nm_new_list = ['_all100d33_','_ZRH_','_all100d33_large_','_ZRH_large_']

for nm, nm_new in zip(nm_list, nm_new_list):
    for cl in ls_cl:
        t = pd.read_pickle('crowdsourced_data_%s_%s_5c.pkl' % (nm,cl))
        t = t.rename(columns={"clust_no": "cl_flag" + nm_new + cl, "labels": "cl_label" + nm_new + cl})
        t2 = t[['ID','x','y','Time','size',"cl_flag" + nm_new + cl, "cl_label" + nm_new + cl]]
        z = z.merge(t2, on=["x","y","Time",'ID','size'], how='left', indicator=False)

z['Flag_CZC'] = 0
z.loc[z['maxCZC_t15_r4']<=35, 'Flag_CZC'] = 1
z.to_pickle('crowd_consolidated_2015-05-02_2023-10-15.pkl')

### Examples of filtering

In [7]:
crowd_data = pd.read_pickle('crowd_consolidated_2015-05-02_2023-10-15.pkl')

Filter the reports according to Barras et al, 2019 (https://doi.org/10.1175/BAMS-D-18-0090.1):

In [9]:
temp = crowd_data.loc[(crowd_data['Flag_30min']==0) & (crowd_data['Flag_blacklist']==0) & (crowd_data['Flag_CZC']==0) &
               ~((crowd_data['size']==0) | (crowd_data['size']==10))]

,ID,day,CustomLocation,x,y,Time,size,CustomTime,SubmissionTime,OsVersion,...,cl_label_all100d33_large_12km_720s,cl_flag_all100d33_large_16km_960s,cl_label_all100d33_large_16km_960s,cl_flag_ZRH_large_8km_480s,cl_label_ZRH_large_8km_480s,cl_flag_ZRH_large_12km_720s,cl_label_ZRH_large_12km_720s,cl_flag_ZRH_large_16km_960s,cl_label_ZRH_large_16km_960s,Flag_CZC
13,97262821-DB21-4159-9449-8A909A8492E1,2015-05-15,False,683416,248333,2015-05-15 07:54:38+00:00,3,False,2015-05-15 07:54:46+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
27,09E914A4-19BE-4178-A0C5-D1933E14C59B,2015-05-15,True,694542,253769,2015-05-15 11:15:55+00:00,2,False,2015-05-15 11:16:08+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
28,09E914A4-19BE-4178-A0C5-D1933E14C59B,2015-05-15,True,691069,236399,2015-05-15 11:16:27+00:00,2,False,2015-05-15 11:16:41+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
57,09E914A4-19BE-4178-A0C5-D1933E14C59B,2015-05-19,True,717339,96360,2015-05-19 13:28:27+00:00,3,False,2015-05-19 13:28:41+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
84,730E8977-63F4-4D4D-AAE8-572E1F2D8F8C,2015-05-26,False,661207,270029,2015-05-26 10:03:47+00:00,1,False,2015-05-26 10:04:03+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347818,dc25a980-d843-4b37-818e-15d934dfa223,2023-10-14,False,501691,127662,2023-10-14 13:01:32.733999872+00:00,12,False,2023-10-14 13:01:45.296999936+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
347819,EEF3BCF1-4A13-4979-B147-7DA10854B6FB,2023-10-14,False,662056,255923,2023-10-14 13:06:24.896000+00:00,12,False,2023-10-14 13:06:30.227000064+00:00,ios-17.0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
347821,D3B28BEB-106D-4620-8B27-91D1DE70C8D8,2023-10-14,False,604088,207339,2023-10-14 13:48:18.900000+00:00,14,False,2023-10-14 13:48:21.061000192+00:00,ios-17.0.3,...,-1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,0
347823,c6a33e6f-465c-465f-8e9e-2c74dcf7b8ab,2023-10-14,True,641534,203376,2023-10-14 14:45:43+00:00,13,True,2023-10-14 14:46:01.864000+00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Further removes reports if same user submitted more than 4 on same day:

In [ ]:
temp = temp.loc[(temp['Flag_N_day_ID_4']==0)]

Clusters were made from reports (excluding the no hail reports) which were already filtered according to Flag_30min, Flag_blacklist, Flag_N_day_ID_4, so those filters don't have to be specified again.

Keep only reports that were part of a cluster on the Swiss100 region, using a 12km distance parameter and 12 minutes time parameter:

In [13]:
temp = crowd_data.loc[(crowd_data['cl_flag_all100d33_12km_720s']==0)]

Further removes reports where maximum reflectivity (CZC) was < 35 dBZ within 4km radius and 15min time window:

In [ ]:
temp = temp.loc[temp['Flag_CZC'] == 0]

Keep only reports of hail larger than 2 cm that were part of a cluster on the ZRH region, using a 8km distance parameter and 8 minutes time parameter:

In [15]:
temp = crowd_data.loc[(crowd_data['cl_flag_ZRH_large_8km_480s']==0)]

2017

Find the number of different clusters on 2021-06-28 on the Swiss100 region, using a 16km distance parameter and 16 minutes time parameter (-1 labels correspond to report that were not part of a cluster)

In [25]:
len(temp.loc[(temp['cl_label_all100d33_16km_960s'] > -1) & (crowd_data['day'] == '2021-06-28'),'cl_label_all100d33_16km_960s'].unique())

10